In [2]:
import pandas as pd
import itertools
import numpy as np

In [1]:
csv= '/home/mei/nas/docker/thesis/data/csv/'

In [16]:
print('==> Loading data from timeseries files...')
timeseries_lab = pd.read_csv(csv + 'timeserieslab.csv', low_memory=False)
timeseries_periodic = pd.read_csv(csv+ 'timeseriesperiodic.csv')

==> Loading data from timeseries files...


In [17]:
print("there are {} patients in the  and {} records in lab table.".format(len(list(timeseries_lab ['patientunitstayid'].unique())),len(timeseries_lab)))
print("there are {} patients in the  and {} records in vital periodic table.".format(len(list(timeseries_periodic ['patientunitstayid'].unique())),len(timeseries_periodic)))


there are 10496 patients in the  and 794119 records in lab table.
there are 10496 patients in the  and 2905327 records in vital periodic table.


In [18]:
def reconfigure_timeseries(timeseries, offset_column, feature_column=None, test=False):
    """
    Reconfigure timeseries data by setting multi-index and pivoting if necessary.
    """
    if test:
        timeseries = timeseries.iloc[:5000]  # Limit for testing
    timeseries.set_index(['patientunitstayid', pd.to_timedelta(timeseries[offset_column], unit='min')], inplace=True)
    timeseries.drop(columns=offset_column, inplace=True)
    if feature_column:
        timeseries = timeseries.pivot_table(columns=feature_column, index=timeseries.index)
    timeseries.index = pd.MultiIndex.from_tuples(timeseries.index, names=['patient', 'time'])
    return timeseries

In [19]:
test=False
print('==> Reconfiguring lab timeseries...')
timeseries_lab = reconfigure_timeseries(timeseries_lab, 'labresultoffset', 'labname', test)
timeseries_lab.columns = timeseries_lab.columns.droplevel()

print('==> Reconfiguring periodic timeseries...')
timeseries_periodic = reconfigure_timeseries(timeseries_periodic, 'observationoffset', test=test)

print('==> Combining data together...')
merged = pd.concat([timeseries_lab, timeseries_periodic], axis=0, sort=False)

==> Reconfiguring lab timeseries...
==> Reconfiguring periodic timeseries...
==> Combining data together...


In [26]:
merged

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                   
252784  -1 days +20:55:00     NaN   NaN      NaN     NaN     NaN         NaN   
        -1 days +21:20:00     0.8   0.4     16.6     4.4     NaN        34.0   
        0 days 00:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 00:39:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
...                           ...   ...      ...     ...     ...         ...   
3347496 0 days 23:12:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:07:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:02:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:32:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:52:00       NaN   NaN      NaN     NaN     NaN         NaN   

                           AST (SGOT)   BUN  Base Excess  FiO2  ...  sao2  \
patient time                                                    ...         
252784  -1 days +20:55:00         NaN   NaN          NaN   NaN  ...   NaN   
        -1 days +21:20:00        24.0  22.0          NaN   NaN  ...   NaN   
        0 days 00:20:00           NaN   NaN        -25.6  28.0  ...   NaN   
        0 days 00:39:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 01:20:00           NaN   NaN          NaN   NaN  ...   NaN   
...                               ...   ...          ...   ...  ...   ...   
3347496 0 days 23:12:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:07:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:02:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:32:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 08:52:00           NaN   NaN          NaN   NaN  ...  98.0   

                           heartrate  respiration  cvp  systemicsystolic  \
patient time                                                               
252784  -1 days +20:55:00        NaN          NaN  NaN               NaN   
        -1 days +21:20:00        NaN          NaN  NaN               NaN   
        0 days 00:20:00          NaN          NaN  NaN               NaN   
        0 days 00:39:00          NaN          NaN  NaN               NaN   
        0 days 01:20:00          NaN          NaN  NaN               NaN   
...                              ...          ...  ...               ...   
3347496 0 days 23:12:00         86.0         16.0  NaN               NaN   
        0 days 23:07:00         88.0         24.0  NaN               NaN   
        0 days 23:02:00         86.0         24.0  NaN               NaN   
        0 days 23:32:00         81.0         19.0  NaN               NaN   
        0 days 08:52:00         91.0          NaN  NaN               NaN   

                           systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                               
252784  -1 days +20:55:00                NaN           NaN  NaN  NaN  NaN  
        -1 days +21:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:20:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 00:39:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                  NaN           NaN  NaN  NaN  NaN  
...                                      ...           ...  ...  ...  ...  
3347496 0 days 23:12:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:07:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:02:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:32:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 08:52:00                  NaN           NaN  NaN  NaN  NaN  

[29534

In [9]:
possible_value_ranges = {
   "temperature": (32,43),        # Temperature
    "sao2": (40, 100),             # SpO2
    "heartrate": (30, 400),        # Heart rate
    "respiration": (0, 60),        # Resp. rate
    "cvp": (0, 20),                # CVP
    
    # "etco2": (0, 60),             # EtCO2
    "systemicsystolic": (40, 300), # BP systolic
    "systemicdiastolic": (20, 150),# BP diastolic
    "systemicmean": (30, 200),     # BP mean
    
}

In [30]:
def filter_vital_signs(data, ranges):
    for column, (min_val, max_val) in ranges.items():
        if column in data.columns:
            data = data[(data[column].isna()) | ((data[column] >= min_val) & (data[column] <= max_val))]
    return data

merged = filter_vital_signs(merged, possible_value_ranges)
print("select valid vital signs")
print("There are {} patients and {} records in the vital periodic table.".format(
    merged.index.get_level_values('patient').nunique(),
    len(merged)
))

select valid vital signs
There are 10496 patients and 2953499 records in the vital periodic table.


In [31]:
merged

-basos  -eos  -lymphs  -monos  -polys  ALT (SGPT)  \
patient time                                                                   
252784  -1 days +20:55:00     NaN   NaN      NaN     NaN     NaN         NaN   
        -1 days +21:20:00     0.8   0.4     16.6     4.4     NaN        34.0   
        0 days 00:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 00:39:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 01:20:00       NaN   NaN      NaN     NaN     NaN         NaN   
...                           ...   ...      ...     ...     ...         ...   
3347496 0 days 23:12:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:07:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:02:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 23:32:00       NaN   NaN      NaN     NaN     NaN         NaN   
        0 days 08:52:00       NaN   NaN      NaN     NaN     NaN         NaN   

                           AST (SGOT)   BUN  Base Excess  FiO2  ...  sao2  \
patient time                                                    ...         
252784  -1 days +20:55:00         NaN   NaN          NaN   NaN  ...   NaN   
        -1 days +21:20:00        24.0  22.0          NaN   NaN  ...   NaN   
        0 days 00:20:00           NaN   NaN        -25.6  28.0  ...   NaN   
        0 days 00:39:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 01:20:00           NaN   NaN          NaN   NaN  ...   NaN   
...                               ...   ...          ...   ...  ...   ...   
3347496 0 days 23:12:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:07:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:02:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 23:32:00           NaN   NaN          NaN   NaN  ...   NaN   
        0 days 08:52:00           NaN   NaN          NaN   NaN  ...  98.0   

                           heartrate  respiration  cvp  systemicsystolic  \
patient time                                                               
252784  -1 days +20:55:00        NaN          NaN  NaN               NaN   
        -1 days +21:20:00        NaN          NaN  NaN               NaN   
        0 days 00:20:00          NaN          NaN  NaN               NaN   
        0 days 00:39:00          NaN          NaN  NaN               NaN   
        0 days 01:20:00          NaN          NaN  NaN               NaN   
...                              ...          ...  ...               ...   
3347496 0 days 23:12:00         86.0         16.0  NaN               NaN   
        0 days 23:07:00         88.0         24.0  NaN               NaN   
        0 days 23:02:00         86.0         24.0  NaN               NaN   
        0 days 23:32:00         81.0         19.0  NaN               NaN   
        0 days 08:52:00         91.0          NaN  NaN               NaN   

                           systemicdiastolic  systemicmean  st1  st2  st3  
patient time                                                               
252784  -1 days +20:55:00                NaN           NaN  NaN  NaN  NaN  
        -1 days +21:20:00                NaN           NaN  NaN  NaN  NaN  
        0 days 00:20:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 00:39:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 01:20:00                  NaN           NaN  NaN  NaN  NaN  
...                                      ...           ...  ...  ...  ...  
3347496 0 days 23:12:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:07:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:02:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 23:32:00                  NaN           NaN  NaN  NaN  NaN  
        0 days 08:52:00                  NaN           NaN  NaN  NaN  NaN  

[29534

In [32]:
def gen_patient_chunk(patients, merged, size=500): # 500 is a good size for LSTM
    """
    Generate patient data chunks for processing.
    """
    it = iter(patients)
    chunk = list(itertools.islice(it, size))
    while chunk:
        yield merged.loc[chunk]
        chunk = list(itertools.islice(it, size))

In [33]:
test=False


def resample_and_mask(timeseries, eICU_path, header, mask_decay=True, decay_rate=4/3, test=False, verbose=False):
    """
    Resample timeseries data to hourly intervals and calculate masking features.
    """
    if verbose:
        print('Resampling to 1 hour intervals...')
    timeseries = timeseries.groupby(level=[0, 1]).mean()
    unstacked = timeseries.unstack(level=0)
    unstacked.index = unstacked.index.ceil(freq='h')
    resampled = unstacked.resample('h', closed='right', label='right').mean()

    if mask_decay:
        if verbose:
            print('Calculating mask decay features...')
        mask_bool = resampled.notnull()
        mask = mask_bool.astype(int)
        mask.replace({0: np.nan}, inplace=True) 
        inv_mask_bool = ~mask_bool
        count_non_measurements = inv_mask_bool.cumsum() - inv_mask_bool.cumsum().where(mask_bool).ffill().fillna(0)
        mask = mask.ffill().fillna(0) / (count_non_measurements * decay_rate).replace(0, 1)
        mask = mask.iloc[-24:]
    else:
        if verbose:
            print('Calculating binary mask features...')
        mask = resampled.iloc[-24:].notnull().astype(int)

    if verbose:
        print('Filling missing data forwards...')
    resampled = resampled.ffill().iloc[-24:]
    resampled.index = list(range(1, 25))
    mask.index = list(range(1, 25))

    if verbose:
        print('Filling in remaining values with zeros...')
    resampled.fillna(0, inplace=True)

    if verbose:
        print('Reconfiguring and combining features with mask features...')
        
    resampled = resampled.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    mask = mask.stack(level=1,future_stack=True).swaplevel(0, 1).sort_index(level=0)
    
    mask.columns = [str(col) + '_mask' for col in mask.columns]
    final = pd.concat([resampled, mask], axis=1)

    if verbose:     # if verbose=True, print the progress
        print('Saving progress...')
    if test==False:  # if test=False, save the progress
        final.to_csv(csv + 'preprocessed_timeseries.csv', mode='a', header=header)

In [34]:
import gc

patients = merged.index.unique(level=0)
gen_chunks = gen_patient_chunk(patients, merged)
header = True
print('==> Initiating main processing loop...')
for i, patient_chunk in enumerate(gen_chunks, start=1):
    final=resample_and_mask(patient_chunk, csv, header, mask_decay=True, decay_rate=4/3, test=test, verbose=False)
    print(f'==> Processed {i * 500} patients...')
    header = False
    
    # Clear memory
    del patient_chunk
    gc.collect()

==> Initiating main processing loop...
==> Processed 500 patients...
==> Processed 1000 patients...
==> Processed 1500 patients...
==> Processed 2000 patients...
==> Processed 2500 patients...
==> Processed 3000 patients...
==> Processed 3500 patients...
==> Processed 4000 patients...
==> Processed 4500 patients...
==> Processed 5000 patients...
==> Processed 5500 patients...
==> Processed 6000 patients...
==> Processed 6500 patients...
==> Processed 7000 patients...
==> Processed 7500 patients...
==> Processed 8000 patients...
==> Processed 8500 patients...
==> Processed 9000 patients...
==> Processed 9500 patients...
==> Processed 10000 patients...
==> Processed 10500 patients...


In [35]:
final=pd.read_csv(csv + 'preprocessed_timeseries.csv')

In [36]:
final

,patient,Unnamed: 1,-basos,-eos,-lymphs,-monos,-polys,ALT (SGPT),AST (SGOT),BUN,...,sao2_mask,heartrate_mask,respiration_mask,cvp_mask,systemicsystolic_mask,systemicdiastolic_mask,systemicmean_mask,st1_mask,st2_mask,st3_mask
0,252784,1,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,1.0,1.0,1.0
1,252784,2,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,1.0,1.0,1.0
2,252784,3,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,1.0,1.0,1.0
3,252784,4,0.8,0.4,16.6,4.4,0.0,34.0,24.0,22.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,1.0,1.0,1.0
4,252784,5,0.8,0.4,16.6,4.4,0.0,34.0,24.0,23.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,1.0,1.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
251899,3347496,20,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251900,3347496,21,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251901,3347496,22,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,1.000,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0
251902,3347496,23,1.0,1.0,15.0,4.0,0.0,13.0,19.0,42.0,...,0.750,1.00,1.00,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [38]:
print("There are {} patients and {} records in the time series table.".format(
    final['patient'].nunique(),
    len(final)
))

There are 10496 patients and 251904 records in the time series table.
